In [1]:
!pip show fpdf

Name: fpdf
Version: 1.7.2
Summary: Simple PDF generation for Python
Home-page: http://code.google.com/p/pyfpdf
Author: Olivier PLATHEY ported by Max
Author-email: maxpat78@yahoo.it
License: LGPLv3+
Location: /Users/george.cherian/Work/WellsFargo/EDIP/LexiGraph/.venv/lib/python3.11/site-packages
Requires: 
Required-by: 


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Function to generate random dates
def generate_random_dates(start_date, end_date, n):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    date_list = [start + timedelta(days=random.randint(0, (end - start).days)) for _ in range(n)]
    return date_list

# Function to generate random transactions
def generate_random_transactions(n):
    descriptions = ['Food', 'Bus', 'Train', 'Mall', 'Grocery', 'Rent', 'Salary', 'Bank interest', 'Electricity', 'Internet', 'Gym']
    transactions = []
    for _ in range(n):
        date = random.choice(dates)
        description = random.choice(descriptions)
        amount = round(random.uniform(10, 500), 2)
        # transaction_type = random.choice(['CR', 'DR'])
        if description in ['Salary', 'Bank interest']:
            transaction_type = 'CR'
        else:
            transaction_type = 'DR'
        transactions.append([date, description, amount, transaction_type])
    return transactions

# Generate random data
num_entries = 100
dates = generate_random_dates('2022-06-01', '2023-06-01', num_entries)
transactions = generate_random_transactions(num_entries)

# Create a DataFrame
df = pd.DataFrame(transactions, columns=['Date', 'Description', 'Amount', 'Type'])

# Sort by date
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date').reset_index(drop=True)

df.head()

/Users/george.cherian/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Date,Description,Amount,Type
0,2022-06-11,Food,238.82,DR
1,2022-06-28,Mall,27.82,DR
2,2022-07-04,Salary,451.56,CR
3,2022-07-13,Grocery,395.58,DR
4,2022-07-24,Rent,43.10,DR


In [3]:
# to create csv format of data
df.to_csv('Bank_Statement.csv', index=False)

In [4]:
!pwd


/Users/george.cherian/Work/WellsFargo/EDIP/LexiGraph/Wells Fargo


In [7]:
# to make pdf of dummy statement
!pip install fpdf
import fpdf

ModuleNotFoundError: No module named 'fpdf'

In [5]:
# Create the updated DataFrame as per the user's code
# Generate random data
num_entries = 100
dates = generate_random_dates('2022-06-01', '2023-06-01', num_entries)
transactions = generate_random_transactions(num_entries)

# Create a DataFrame
df = pd.DataFrame(transactions, columns=['Date', 'Description', 'Amount', 'Type'])

# Sort by date
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date').reset_index(drop=True)

# Function to generate PDF
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Bank Statement', 0, 1, 'C')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

    def table(self, data):
        self.set_font('Arial', 'B', 10)
        col_width = self.w / 4  # distribute content evenly
        th = self.font_size
        
        # Table header
        self.cell(col_width, th, 'Date', border=1)
        self.cell(col_width, th, 'Description', border=1)
        self.cell(col_width, th, 'Amount', border=1)
        self.cell(col_width, th, 'Type', border=1)
        self.ln(th)
        
        # Table body
        self.set_font('Arial', '', 10)
        for row in data.itertuples():
            self.cell(col_width, th, str(row.Date.date()), border=1)
            self.cell(col_width, th, row.Description, border=1)
            self.cell(col_width, th, f'{row.Amount:.2f}', border=1)
            self.cell(col_width, th, row.Type, border=1)
            self.ln(th)

# Create PDF
pdf = PDF()
pdf.add_page()
pdf.table(df)

# Save the PDF to a file
output_path = "Bank_Statement.pdf"
pdf.output(output_path)

output_path

ModuleNotFoundError: No module named 'fpdf'

In [8]:
import sys, io
from PIL import Image
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

def main(path):
    # Load (PDF or image). For PDF, use first page for a quick test
    if path.lower().endswith(".pdf"):
        doc = DocumentFile.from_pdf(path)
        doc = DocumentFile.from_images(doc.render()[0])  # first page raster
    else:
        img = Image.open(path).convert("RGB")
        buf = io.BytesIO(); img.save(buf, format="PNG")
        doc = DocumentFile.from_images(buf.getvalue())

    model = ocr_predictor(pretrained=True)  # downloads weights on first run
    res = model(doc)                        # run OCR
    # Print first 5 lines
    lines = []
    for p in res.pages:
        for b in p.blocks:
            for l in b.lines:
                lines.append(" ".join([w.value for w in l.words]))
    print(f"Lines detected: {len(lines)}")
    for t in lines[:5]:
        print(" -", t)

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python ocr_smoke.py <image_or_pdf>")
        sys.exit(1)
    main(sys.argv[1])

ModuleNotFoundError: No module named 'doctr'